In [8]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_hchang13" #please modify so you store data in your collection
# beginning page index
begin = "0"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]
my_char='h'


beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

source_url = "https://sourceforge.net/directory/?q=" + my_char + "&sort=name&page="
rest_url = "https://sourceforge.net/rest/p/"

gleft = 0

header = {'per_page': 99}

def project_exists(url):
    r = requests.get(url)
    if r.status_code == 200:
        return True
    return False

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

def get_source(url, coll, rest):
    page = 1
    project_count = 0
    while True:
        resp = requests.get(url + str(page))
        text = resp.text
        soup = BeautifulSoup(text, 'html.parser')
        if re.search('No results found.', soup.get_text()):
            return

        for link in soup.find_all(class_="project-icon", href=True):
            name = re.findall('/projects/([A-Za-z0-9\-]*)', link.get('href'))
            name = name[0] if name else None
            if name is not None and name.lower().startswith(my_char):
                resp = requests.get(rest + name)
                if resp.status_code == 200:
                    info = json.loads(resp.text)
                    info['forge'] = 'sourceforge'
                    print(info)
                    try:
                        coll.insert_one(info)
                    except pymongo.errors.DuplicateKeyError:
                        continue
                    project_count += 1
                    if project_count >= 50:
                        print('Finish!')
                        return
        page += 1
    return

# send queries and extract urls 
def get(url, coll):

    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    project_count = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):
            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            print('Start!')
            for el in array:
                if el['name'].lower().startswith(my_char):
                    if project_exists(el['http_url_to_repo']):
                        project_count += 1
                        el['forge'] = 'gitlab'
                        coll.insert_one(el)
                        if project_count >= 50:
                            return
                
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if el['name'].lower().startswith(my_char):
                                if project_exists(el['http_url_to_repo']):
                                    print(el)
                                    project_count += 1
                                    el['forge'] = 'gitlab'
                                    coll.insert_one(el)
                                    if project_count >= 50:
                                        return
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
        
#start retrieving        
get(beginurl,coll)
get_source(source_url, coll, rest_url)

Start!
{'ssh_url_to_repo': 'git@gitlab.com:fanguloa/hotel_restaurant.git', 'description': '', 'forks_count': 0, 'path_with_namespace': 'fanguloa/hotel_restaurant', 'star_count': 0, 'web_url': 'https://gitlab.com/fanguloa/hotel_restaurant', 'readme_url': 'https://gitlab.com/fanguloa/hotel_restaurant/blob/master/README.rst', 'name': 'hotel_restaurant', 'http_url_to_repo': 'https://gitlab.com/fanguloa/hotel_restaurant.git', 'created_at': '2018-11-29T20:38:47.640Z', 'namespace': {'kind': 'user', 'parent_id': None, 'id': 3487998, 'path': 'fanguloa', 'name': 'fanguloa', 'full_path': 'fanguloa'}, 'name_with_namespace': 'Felipe Angulo Achurra / hotel_restaurant', 'avatar_url': None, 'default_branch': 'master', 'id': 9688149, 'path': 'hotel_restaurant', 'last_activity_at': '2018-11-29T20:38:47.640Z', 'tag_list': []}
{'ssh_url_to_repo': 'git@gitlab.com:fanguloa/hotel_housekeeping.git', 'description': '', 'forks_count': 0, 'path_with_namespace': 'fanguloa/hotel_housekeeping', 'star_count': 0, 'we

{'ssh_url_to_repo': 'git@gitlab.com:nahidnsu152/Hospital-Management-System.git', 'description': None, 'forks_count': 0, 'path_with_namespace': 'nahidnsu152/Hospital-Management-System', 'star_count': 0, 'web_url': 'https://gitlab.com/nahidnsu152/Hospital-Management-System', 'readme_url': None, 'name': 'Hospital-Management-System', 'http_url_to_repo': 'https://gitlab.com/nahidnsu152/Hospital-Management-System.git', 'created_at': '2018-11-29T18:11:11.281Z', 'namespace': {'kind': 'user', 'parent_id': None, 'id': 4029526, 'path': 'nahidnsu152', 'name': 'nahidnsu152', 'full_path': 'nahidnsu152'}, 'name_with_namespace': 'Nazmul Hasan Nahid / Hospital-Management-System', 'avatar_url': None, 'default_branch': 'master', 'id': 9686131, 'path': 'Hospital-Management-System', 'last_activity_at': '2018-11-29T18:11:11.281Z', 'tag_list': []}
{'ssh_url_to_repo': 'git@gitlab.com:navigaid/hello.git', 'description': '', 'forks_count': 0, 'path_with_namespace': 'navigaid/hello', 'star_count': 0, 'web_url': 

{'ssh_url_to_repo': 'git@gitlab.com:bryandv/hackthefuture-code.git', 'description': '-Make a folder with your team-name\r\n-Put all your solutions in that ', 'forks_count': 0, 'path_with_namespace': 'bryandv/hackthefuture-code', 'star_count': 0, 'web_url': 'https://gitlab.com/bryandv/hackthefuture-code', 'readme_url': 'https://gitlab.com/bryandv/hackthefuture-code/blob/master/README.md', 'name': 'HackTheFuture-Code', 'http_url_to_repo': 'https://gitlab.com/bryandv/hackthefuture-code.git', 'created_at': '2018-11-29T14:50:49.201Z', 'namespace': {'kind': 'user', 'parent_id': None, 'id': 4153604, 'path': 'bryandv', 'name': 'bryandv', 'full_path': 'bryandv'}, 'name_with_namespace': 'bryandv / HackTheFuture-Code', 'avatar_url': None, 'default_branch': 'master', 'id': 9683153, 'path': 'hackthefuture-code', 'last_activity_at': '2018-11-29T14:50:49.201Z', 'tag_list': []}
{'ssh_url_to_repo': 'git@gitlab.com:KlossyKodeKamp/hello-world-ruby-v-000.git', 'description': '', 'forks_count': 0, 'path_wi

{'ssh_url_to_repo': 'git@gitlab.com:MKoppJones/haversack.git', 'description': '', 'forks_count': 0, 'path_with_namespace': 'MKoppJones/haversack', 'star_count': 0, 'web_url': 'https://gitlab.com/MKoppJones/haversack', 'readme_url': 'https://gitlab.com/MKoppJones/haversack/blob/master/README.md', 'name': 'haversack', 'http_url_to_repo': 'https://gitlab.com/MKoppJones/haversack.git', 'created_at': '2018-11-29T13:15:46.235Z', 'namespace': {'kind': 'user', 'parent_id': None, 'id': 1041239, 'path': 'MKoppJones', 'name': 'MKoppJones', 'full_path': 'MKoppJones'}, 'name_with_namespace': 'Michael Kopp-Jones / haversack', 'avatar_url': None, 'default_branch': 'master', 'id': 9681563, 'path': 'haversack', 'last_activity_at': '2018-11-29T13:15:46.235Z', 'tag_list': []}
{'ssh_url_to_repo': 'git@gitlab.com:willardshikami/hit-fe.git', 'description': 'Humans in Tech', 'forks_count': 0, 'path_with_namespace': 'willardshikami/hit-fe', 'star_count': 0, 'web_url': 'https://gitlab.com/willardshikami/hit-fe

{'ssh_url_to_repo': 'git@gitlab.com:leo.costa/hessian-integration.git', 'description': 'API de serviĂ§os que conversam entre si', 'forks_count': 0, 'path_with_namespace': 'leo.costa/hessian-integration', 'star_count': 0, 'web_url': 'https://gitlab.com/leo.costa/hessian-integration', 'readme_url': 'https://gitlab.com/leo.costa/hessian-integration/blob/master/README.md', 'name': 'hessian-integration', 'http_url_to_repo': 'https://gitlab.com/leo.costa/hessian-integration.git', 'created_at': '2018-11-29T07:55:36.550Z', 'namespace': {'kind': 'user', 'parent_id': None, 'id': 3594086, 'path': 'leo.costa', 'name': 'leo.costa', 'full_path': 'leo.costa'}, 'name_with_namespace': 'Leo Jose da Costa / hessian-integration', 'avatar_url': None, 'default_branch': 'master', 'id': 9673576, 'path': 'hessian-integration', 'last_activity_at': '2018-11-29T07:55:36.550Z', 'tag_list': []}
{'ssh_url_to_repo': 'git@gitlab.com:m.moradi.ps/herfesaaz.git', 'description': '', 'forks_count': 0, 'path_with_namespace'

...

{'status': 'active', 'tools': [{'mount_point': 'discussion', 'icons': {'24': 'images/forums_24.png', '32': 'images/forums_32.png', '48': 'images/forums_48.png'}, 'url': '/p/hbhidtest1/discussion/', 'mount_label': 'Discussion', 'installable': True, 'name': 'discussion', 'tool_label': 'Discussion'}, {'mount_point': 'wiki', 'icons': {'24': 'images/wiki_24.png', '32': 'images/wiki_32.png', '48': 'images/wiki_48.png'}, 'url': '/p/hbhidtest1/wiki/', 'mount_label': 'Wiki', 'installable': True, 'name': 'wiki', 'tool_label': 'Wiki'}, {'mount_point': 'files', 'icons': {'24': 'images/downloads_24.png', '32': 'images/downloads_32.png', '48': 'images/downloads_48.png'}, 'url': '/p/hbhidtest1/files/', 'mount_label': 'Files', 'installable': False, 'name': 'files', 'tool_label': 'Files'}, {'mount_point': 'code', 'icons': {'24': 'images/code_24.png', '32': 'images/code_32.png', '48': 'images/code_48.png'}, 'url': '/p/hbhidtest1/code/', 'mount_label': 'Code', 'installable': True, 'name': 'git', 'tool_la

{'status': 'active', 'tools': [{'mount_point': 'files', 'icons': {'24': 'images/downloads_24.png', '32': 'images/downloads_32.png', '48': 'images/downloads_48.png'}, 'url': '/p/hexpand/files/', 'mount_label': 'Files', 'installable': False, 'name': 'files', 'tool_label': 'Files'}, {'mount_point': 'news', 'icons': {'24': 'images/blog_24.png', '32': 'images/blog_32.png', '48': 'images/blog_48.png'}, 'url': '/p/hexpand/news/', 'mount_label': 'News', 'installable': True, 'name': 'blog', 'tool_label': 'Blog'}, {'mount_point': 'bugs', 'icons': {'24': 'images/tickets_24.png', '32': 'images/tickets_32.png', '48': 'images/tickets_48.png'}, 'url': '/p/hexpand/bugs/', 'mount_label': 'Bugs', 'installable': True, 'name': 'tickets', 'tool_label': 'Tickets'}, {'mount_point': 'code', 'icons': {'24': 'images/code_24.png', '32': 'images/code_32.png', '48': 'images/code_48.png'}, 'url': '/p/hexpand/code/', 'mount_label': 'Code', 'installable': False, 'name': 'cvs', 'tool_label': 'CVS'}, {'mount_point': 's

{'status': 'moved', 'tools': [{'mount_point': 'reviews', 'icons': {'24': 'images/sftheme/24x24/blog_24.png', '32': 'images/sftheme/32x32/blog_32.png', '48': 'images/sftheme/48x48/blog_48.png'}, 'url': '/p/hirbtextracter/reviews/', 'mount_label': 'Reviews', 'installable': False, 'name': 'reviews', 'tool_label': 'Reviews'}, {'mount_point': 'wiki', 'icons': {'24': 'images/wiki_24.png', '32': 'images/wiki_32.png', '48': 'images/wiki_48.png'}, 'url': '/p/hirbtextracter/wiki/', 'mount_label': 'Wiki', 'installable': True, 'name': 'wiki', 'tool_label': 'Wiki'}, {'mount_point': 'files', 'icons': {'24': 'images/downloads_24.png', '32': 'images/downloads_32.png', '48': 'images/downloads_48.png'}, 'url': '/p/hirbtextracter/files/', 'mount_label': 'Files', 'installable': False, 'name': 'files', 'tool_label': 'Files'}, {'url': '/p/hirbtextracter/summary/', 'icons': {'24': 'images/sftheme/24x24/blog_24.png', '32': 'images/sftheme/32x32/blog_32.png', '48': 'images/sftheme/48x48/blog_48.png'}, 'mount_p

{'status': 'abandoned', 'tools': [{'mount_point': 'code', 'icons': {'24': 'images/code_24.png', '32': 'images/code_32.png', '48': 'images/code_48.png'}, 'url': '/p/h-m-c/code/', 'mount_label': 'Code', 'installable': True, 'name': 'git', 'tool_label': 'Git'}, {'mount_point': 'tickets', 'icons': {'24': 'images/tickets_24.png', '32': 'images/tickets_32.png', '48': 'images/tickets_48.png'}, 'url': '/p/h-m-c/tickets/', 'mount_label': 'Tickets', 'installable': True, 'name': 'tickets', 'tool_label': 'Tickets'}, {'mount_point': 'discussion', 'icons': {'24': 'images/forums_24.png', '32': 'images/forums_32.png', '48': 'images/forums_48.png'}, 'url': '/p/h-m-c/discussion/', 'mount_label': 'Discussion', 'installable': True, 'name': 'discussion', 'tool_label': 'Discussion'}, {'url': '/p/h-m-c/summary/', 'icons': {'24': 'images/sftheme/24x24/blog_24.png', '32': 'images/sftheme/32x32/blog_32.png', '48': 'images/sftheme/48x48/blog_48.png'}, 'mount_point': 'summary', 'mount_label': 'Summary', 'installa

{'status': 'active', 'tools': [{'mount_point': 'files', 'icons': {'24': 'images/downloads_24.png', '32': 'images/downloads_32.png', '48': 'images/downloads_48.png'}, 'url': '/p/h-child-abuse/files/', 'mount_label': 'Files', 'installable': False, 'name': 'files', 'tool_label': 'Files'}, {'mount_point': 'wiki', 'icons': {'24': 'images/wiki_24.png', '32': 'images/wiki_32.png', '48': 'images/wiki_48.png'}, 'url': '/p/h-child-abuse/wiki/', 'mount_label': 'Wiki', 'installable': True, 'name': 'wiki', 'tool_label': 'Wiki'}, {'url': '/p/h-child-abuse/summary/', 'icons': {'24': 'images/sftheme/24x24/blog_24.png', '32': 'images/sftheme/32x32/blog_32.png', '48': 'images/sftheme/48x48/blog_48.png'}, 'mount_point': 'summary', 'mount_label': 'Summary', 'installable': False, 'sourceforge_group_id': 119562, 'name': 'summary', 'tool_label': 'Summary'}, {'mount_point': 'support', 'icons': {'24': 'images/sftheme/24x24/blog_24.png', '32': 'images/sftheme/32x32/blog_32.png', '48': 'images/sftheme/48x48/blog